## Código

Veamos cómo podríamos implementar el análisis discriminante lineal desde cero utilizando Python

En el tutorial siguiente, trabajaremos con el conjunto de datos de vino que se puede obtener del repositorio de aprendizaje automático UCI. Afortunadamente, la biblioteca "scitkit-learn" proporciona los datos


In [1]:
## Librerias Y cargamos la base de datos

from sklearn.datasets import load_wine
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# importing or loading the dataset 
dataset = pd.read_csv('breast-cancer-wisconsin.csv') 
  
# distributing the dataset into two components X and Y 
X = dataset.iloc[:, 0:10] #Variables de entrada (Caracteristicas de cada vino)
y = dataset.iloc[:, 10] #Variable de salida (tipo de vino)

"""
wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = pd.Categorical.from_codes(wine.target, wine.target_names)
"""
print(type(X))

<class 'pandas.core.frame.DataFrame'>


In [2]:
# Creamos un DataFrame que contiene tanto las características como las clases.
df = X.join(pd.Series(y, name='class'))
print(df)

     1000025   5   1  1.1  1.2  2  1.3   3  1.4  1.5  class
0    1002945   5   4    4    5  7   10   3    2    1      2
1    1015425   3   1    1    1  2    2   3    1    1      2
2    1016277   6   8    8    1  3    4   3    7    1      2
3    1017023   4   1    1    3  2    1   3    1    1      2
4    1017122   8  10   10    8  7   10   9    7    1      4
5    1018099   1   1    1    1  2   10   3    1    1      2
6    1018561   2   1    2    1  2    1   3    1    1      2
7    1033078   2   1    1    1  2    1   1    1    5      2
8    1033078   4   2    1    1  2    1   2    1    1      2
9    1035283   1   1    1    1  1    1   3    1    1      2
10   1036172   2   1    1    1  2    1   2    1    1      2
11   1041801   5   3    3    3  2    3   4    4    1      4
12   1043999   1   1    1    1  2    3   3    1    1      2
13   1044572   8   7    5   10  7    9   5    5    4      4
14   1047630   7   4    6    4  6    1   4    3    1      4
15   1048672   4   1    1    1  2    1  

In [3]:
# Para cada clase, creamos un vector con las medias de cada característica

class_feature_means = pd.DataFrame()
for c, rows in df.groupby('class'):
    class_feature_means[c] = rows.mean()
class_feature_means


,2,4
1000025,1.107826e+06,1.003505e+06
5,2.951860e+00,7.195021e+00
1,1.326039e+00,6.572614e+00
1.1,1.444201e+00,6.560166e+00
1.2,1.365427e+00,5.547718e+00
2,2.120350e+00,5.298755e+00
1.3,1.336980e+00,7.572614e+00
3,2.098468e+00,5.979253e+00
1.4,1.291028e+00,5.863071e+00
1.5,1.063457e+00,2.589212e+00


In [4]:
#Luego, conectamos los "mean vectors" (mi) 
#para obtener la matriz de dispersión dentro de la clase.

within_class_scatter_matrix = np.zeros((13,13))
for c, rows in df.groupby('class'):
    rows = rows.drop(['class'], axis=1)
    s = np.zeros((13,13))
    
for index, row in rows.iterrows():
        x, mc = row.values.reshape(13,1), class_feature_means[c].values.reshape(13,1)
        s += (x - mc).dot((x - mc).T)
        within_class_scatter_matrix += s


# A continuación, calculamos la matriz de dispersión entre clases 

feature_means = df.mean()
between_class_scatter_matrix = np.zeros((13,13))
for c in class_feature_means:    
    n = len(df.loc[df['class'] == c].index)
    
    mc, m = class_feature_means[c].values.reshape(13,1), feature_means.values.reshape(13,1)
    
    between_class_scatter_matrix += n * (mc - m).dot((mc - m).T)
    
    
    
# Luego, resolvemos el problema del valor propio generalizado para obtener los discriminantes lineales

eigen_values, eigen_vectors = np.linalg.eig(np.linalg.inv(within_class_scatter_matrix).dot(between_class_scatter_matrix))


ValueError: cannot reshape array of size 10 into shape (13,1)

Los vectores propios con los valores propios más altos llevan la mayor cantidad de información sobre la distribución de los datos. Por lo tanto, clasificamos los valores propios de mayor a menor y seleccionamos los primeros k vectores propios. Con el fin de garantizar que el valor propio se asigne al mismo vector propio después de la clasificación, los colocamos en una matriz temporal.

In [ ]:
pairs = [(np.abs(eigen_values[i]), eigen_vectors[:,i]) for i in range(len(eigen_values))]
pairs = sorted(pairs, key=lambda x: x[0], reverse=True)
for pair in pairs:
    print(pair[0])
    


In [ ]:
#Primero, creamos una matriz W con los dos primeros vectores propios.
w_matrix = np.hstack((pairs[0][1].reshape(13,1), pairs[1][1].reshape(13,1))).real

Luego, guardamos el producto escalar de X y W en una nueva matriz Y.

Donde X es una matriz n × d con n muestras y d dimensiones, e Y es una matriz n × k con n muestras y dimensiones k (k <n), en otras palabras, Y se compone de los componentes LDA, o dicho de otra manera, el nuevo espacio de características.

In [ ]:
X_lda = np.array(X.dot(w_matrix))
le = LabelEncoder()
y = le.fit_transform(df['class'])

#Luego, graficamos los datos en función de los dos componentes LDA y usamos un color diferente para cada clase.

plt.xlabel('LD1')
plt.ylabel('LD2')
plt.scatter(
    X_lda[:,0],
    X_lda[:,1],
    c=y,
    cmap='rainbow',
    alpha=0.7,
    edgecolors='b'
)


In [ ]:
#Usamos la clase “LinearDiscriminantAnalysis” disponible en la librería “scikit-learn”

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
X_lda = lda.fit_transform(X, y)

# Podemos acceder a la siguiente propiedad para obtener la variance explained" pora cada componente.

lda.explained_variance_ratio_

In [ ]:
# Al igual que antes, "ploteamos" los dos componentes LDA.

plt.xlabel('LD1')
plt.ylabel('LD2')
plt.scatter(
    X_lda[:,0],
    X_lda[:,1],
    c=y,
    cmap='rainbow',
    alpha=0.7,
    edgecolors='b'
)

A continuación, veamos si podemos crear un modelo para clasificar las características LDA, dividimos los datos en conjuntos de entrenamiento y prueba.

Luego, construimos y entrenamos un ”Decision Tree”. Después de predecir la categoría de cada muestra en el conjunto de pruebas, creamos una matriz de confusión para evaluar el rendimiento del modelo.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_lda, y, random_state=1)

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
confusion_matrix(y_test, y_pred)


Para comprara un poco y ver que los resultados son relevantes usamos el Análisis de componentes principales o PCA

Podemos ver que PCA seleccionó los componentes que darían lugar a la mayor difusión (retener la mayor cantidad de información) y no necesariamente los que maximizan la separación entre clases.


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X, y)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.scatter(
    X_pca[:,0],
    X_pca[:,1],
    c=y,
    cmap='rainbow',
    alpha=0.7,
    edgecolors='b'
)